In [1]:
from pathlib import Path
from collections import namedtuple
import pandas as pd
import numpy as np
from numpy import random
import shutil
import os
import sys
import traceback
import re
from pydub import AudioSegment
import moviepy.editor as mp
from scipy.io.wavfile import read
import ftransc
from datetime import datetime
import moviepy.editor as mp

# Define utility functions

# Rename WhatsAPP audio files to something serial

In [2]:
WD = Path.cwd().parents[0]
DATA = WD.joinpath('data')
DATA_COLLECTION = WD.joinpath('audio-data-collection')
AUDIOS_FROM_KAPALAMULA = WD.joinpath('chichewa-docs', 'AudioRecording')
AUD_FILES_SHORT = DATA_COLLECTION.joinpath('short-audio-without-transcripts')
OUTPUT_AUDIO_DIR = DATA.joinpath('chichewa', 'dmatekenya_chichewa_dataset')
district_file= WD.parents[1].joinpath('WBG/malawi/2018Malawi_EAs/2018_Malawi_Enumeration_Areas_With_Data.csv')
APPLE_AUDIOS_DIR = Path('/Users/dmatekenya/Library/Application Support/com.apple.voicememos/Recordings')
COMMON_AUD_EXTENSIONS = ['.aac', '.mp3', '.mp4', '.m4a', '.wav']

In [3]:
def rename_and_unlabelled_audios(input_dir, output_dir):
    """
    Takes  audio files which will not be transcribed,
    renames them, convert 
    to wav and copy to output directory 
    """
    nums = []
    for file in output_dir.iterdir():
        try:
            num = re.findall(r'\d+', file)[0]
            nums.append(num)
        except:
            pass
    if nums:
        start_num = max(current_num_lst) + 1
    else:
        start_num = 1
    
    for au in input_dir.iterdir():
        if au.suffix in COMMON_AUD_EXTENSIONS:
            au_filename_wav = "unlabelled_audio_{}.wav".format(start_num)
            au_dst = output_dir.joinpath(au_filename_wav)
            # convert the audio file
            AudioSegment.from_file(au).export(au_dst, format="wav")
            start_num += 1
    print('Done with renaming and copying')

In [4]:
def get_and_proceess_apple_voice_memos(outdir, start_date=None, end_date=None):
    files = [f for f in APPLE_AUDIOS_DIR.iterdir()]
    cnt = 289
    for f in files:
        try:
            date_str = f.parts[-1][:8]
            date = datetime.strptime(date_str, "%Y%m%d")
            if date >= start_date and date <= end_date:
                print(f)
                # convert to wav
                outf = outdir.joinpath("dunstan_self_recorded_{}.wav".format(cnt))
                AudioSegment.from_file(f).export(outf, format="wav")
                cnt += 1
        except Exception as e:
            print(e)
            print(f.parts[-1])
    print('Copied {} audios files'.format(cnt))

In [5]:
def convert_vid_to_wav(vids_dir, outdir, file_cnt_start=None):
    cnt = file_cnt_start
    for file in vids_dir.iterdir():
        if file.suffix == '.mp4':
            outf = str(outdir.joinpath('aud_from_vid{}.wav'.format(cnt)))
            clip = mp.VideoFileClip(str(file))
            clip.audio.write_audiofile(outf, bitrate='3000k')
            cnt += 1

In [ ]:
outdir_dunstan_auds = DATA_COLLECTION.joinpath('temp-holding-before-sync', 'dunstan-self-records-extra')
start = datetime(2022, 4, 28)
end =  datetime(2022, 4, 30)
get_and_proceess_apple_voice_memos(outdir=outdir_dunstan_auds, start_date=start, 
                                   end_date=end)

In [ ]:
outdir_whatsapp_auds = DATA_COLLECTION.joinpath('temp-holding-before-sync',
                                                'audios-from-whatsapp-extra')
convert_vid_to_wav(vids_dir=Path("/Users/dmatekenya/Downloads/whatsvids/"), 
                   outdir=outdir_whatsapp_auds, file_cnt_start=28)

In [5]:
start_date

datetime.datetime(2022, 4, 29, 0, 0)

In [86]:
tmp = Path("../audio-data-collection/temp-holding-before-sync/from-whatsapp/")

cnt = 1
for f in tmp.iterdir():
    if f.suffix in COMMON_AUD_EXTENSIONS:
        outf = tmp.parents[1].joinpath("aud_from_whatsapp{}.wav".format(cnt))
        AudioSegment.from_file(f).export(outf, format="wav")
        cnt += 1

In [73]:
x = [file for file in outdir_dunstan_auds.iterdir()]

In [4]:
def rename_short_audios():
    
    prop_named_files = [i for i in list(AUD_FILES_SHORT.iterdir()) if i.parts[-1][:3] == 'WAU']
    needs_renaming = [i for i in list(AUD_FILES_SHORT.iterdir()) if i.parts[-1][:3] != 'WAU']
    
    current_num_lst = []
    
    for file in prop_named_files:
        fname = file.parts[-1]
        nums = re.findall(r'\d+', fname)
        if len(nums) == 1:
            current_num_lst.append(int(nums[0]))
        elif len(nums) == 2:
            current_num_lst.append(int(nums[0]))
        else:
            print('Something isnt right')
    
    start_num = max(current_num_lst) + 1
    
    suffixes = []
    for i in needs_renaming:
        suffixes.append(i.suffix)
    suffixes = set(suffixes)
    
    
    cnt = start_num
    for f in needs_renaming:
        if f.suffix in suffixes:
            suf = f.suffix
            new_fpath = AUD_FILES_SHORT.joinpath('WAU{}{}'.format(cnt, suf))
            shutil.move(f, new_fpath)
            cnt += 1
    
    print('Done with renaming')

In [5]:
# rename_short_audios()

## Rename and covert long audios untranscribed audios

In [7]:
in_folder = DATA_COLLECTION.joinpath("untranscribed_audios")
out_folder =  DATA.joinpath('chichewa', 'untranscribed_audios')
rename_and_unlabelled_audios(input_dir=in_folder, output_dir=out_folder)

Done with renaming and copying


In [8]:
tot = 0
for file in out_folder.iterdir():
    audio_seg = AudioSegment.from_wav(file)
    tot += len(audio_seg)//1000

In [10]:
tot/3600

12.3175

# Consolidate Data
Collect all audios and transcripts into one folder and create CSV of audio files

In [6]:
# global variables
SEX = {'Male':0.5, 'Female':0.4, 'Both':0.1}
AGE = ['3-5', '6-11', '12-17', '18-23', '24-29', '30-35', '36-41', 
       '42-47', '48-53', '54-59', '60-65', '65+']
AGE_WITH_PROB = {'3-5':0, '6-11':0, '24-29':0.3, '30-35':0.3, '36-41':0.2, '42-47':0.1, '48-53':0.1}
dfd = pd.read_csv(district_file)
DISTRICTS = {d: 1/dfd.DIST_NAME.nunique() for d in list(dfd.DIST_NAME.unique())}
DISTRICTS_WITH_PROB = {'Lilongwe': 0.3, 'Zomba': 0.4, 'Nkhotakota': 0.1, 'Mchinji': 0.1, 'Ntchisi': 0.10}

In [7]:
whatsapp_audios  = DATA_COLLECTION.joinpath('short-audio-with-transcripts')
long_audios = DATA_COLLECTION.joinpath('long-audio-with-transcripts')
audio_from_sentences = DATA_COLLECTION.joinpath('audio-from-sentences')
csv_sentences = DATA_COLLECTION.joinpath('ChichewaSentences.csv')
tmp_au_without_transcripts = DATA_COLLECTION.joinpath('audio-with-no-transcripts')

In [9]:
## process whatsapp audios
AUDIO_SUFFIXES = set([i.suffix for i in whatsapp_audios.iterdir() if i.suffix != '.txt'] + 
                             [i.suffix for i in long_audios.iterdir() if i.suffix != '.txt'] +
                             [i.suffix for i in AUDIOS_FROM_KAPALAMULA.iterdir() if i.suffix != '.csv'])
AUDIO_SUFFIXES =  AUDIO_SUFFIXES.difference(['.txt'])

In [ ]:
def process_sentences(sentence_file, outdir):
    """
    Extract sentences from CSV, save into separate text file
    """
    # save each row of sentencee as a separate text file
    df = pd.read_csv(sentence_file)
    audio_files = [i.parts[-1].split(".")[0] for i in AUDIOS_FROM_KAPALAMULA.iterdir() if i.suffix in AUDIO_SUFFIXES]
    
    for file in AUDIOS_FROM_KAPALAMULA.iterdir():
        au = file.parts[-1].split(".")[0]
        aud_filename = file.parts[-1]
        
        # copy audio file to outdir
        shutil.copy(file, outdir.joinpath(aud_filename))
        try:
            sentence = df.query('AudioFilename == @au')['Sentence'].iloc[0]
            txt_filename = '{}.txt'.format(au)
            outfile = str(outdir.joinpath(txt_filename))
            f = open(outfile, "a")
            f.write(sentence)
            f.close()
        except Exception as e:
            print(e)
            pass

In [ ]:
def process_folder(indir):
    """
    Move files from the holding audio-data-collection folder to the main data folder. 
    Also convert all audio files to WAV.
    """
    for au in indir.iterdir():
        if au.suffix in AUDIO_SUFFIXES:
            au_filename = au.parts[-1]
            txt_fname = "{}.txt".format(au_filename.split(".")[0])
            txt_fpath = indir.joinpath(txt_fname)
            if not txt_fpath.exists():
                print('This audio has no corresponding transcription ...')
                shutil.copy(au, tmp_au_without_transcripts.joinpath(au_filename))
                continue
            txt_dest_fpath = OUTPUT_AUDIO_DIR.joinpath(txt_fname)
            if txt_dest_fpath.exists():
                continue
            if au.suffix == '.wav':
                # just copy both WAV and text file to main data directory                           
                dst_au = OUTPUT_AUDIO_DIR.joinpath(au_filename)
                shutil.copy(au, dst_au)
                if txt_fpath.exists():
                    shutil.copy(txt_fpath, txt_dest_fpath)
            else:
                au_filename_wav = "{}.wav".format(au_filename.split(".")[0])
                au_dst = OUTPUT_AUDIO_DIR.joinpath(au_filename_wav)
                # convert the audio file
                AudioSegment.from_file(au).export(au_dst, format="wav")
                # copy text file
                if txt_fpath.exists():
                    shutil.copy(txt_fpath, txt_dest_fpath)

In [ ]:
# process short audio 
process_folder(indir=whatsapp_audios)

In [ ]:
# process long_audios 
process_folder(indir=long_audios)

In [ ]:
# process audios from sentences
process_sentences(csv_sentences, outdir=audio_from_sentences)

In [ ]:
process_folder(indir=audio_from_sentences)

In [70]:
cols = {'speaker_id': None, 'gender': None, 'age': None, 	'dialect':None,
        'audio_filename': None, 'transc_filename': None, 'collection_mode': None, 'num_speakers': None}

In [136]:
def document_the_data():
    
    data = []
    for file in OUTPUT_AUDIO_DIR.iterdir():
        if file.suffix == '.wav':
            aud_fname = file.parts[-1]
            audio_seg = AudioSegment.from_wav(file)
            duration_sec = len(audio_seg)//1000
            txt_fname = '{}.txt'.format(aud_fname.split(".")[0])
            if OUTPUT_AUDIO_DIR.joinpath(txt_fname).exists():
                filename = file.parts[-1]
                if filename[:3] == 'WAU':
                    mode = 'Spontaneous'
                    age = np.NaN
                    num_speakers = '1-2'
                elif filename[:2] == 'AU':
                    mode = 'Elicitation through sentence'
                    age = '36-41'
                    num_speakers = '1'
                elif filename[:4] == 'Long':
                    mode = 'Focus group discussion'
                    age = '36-41'
                    num_speakers = '2-4'
                else:
                    mode = 'Spontaneous'
                    age = np.NaN
                    num_speakers = '1-2'
                sex = random.choice(a=list(SEX.keys()), size=1, p=list(SEX.values()))[0]
                age = random.choice(a=list(AGE_WITH_PROB.keys()), size=1, p=list(AGE_WITH_PROB.values()))[0]
                dist =  random.choice(a=list(DISTRICTS_WITH_PROB.keys()), size=1,
                                                p=list(DISTRICTS_WITH_PROB.values()))[0]
                cols = {'age': age,
                        'audio_filename': aud_fname,'duration_sec': duration_sec,
                        'transc_filename': txt_fname,
                        'collection_mode': mode, 'num_speakers': num_speakers
                       }
                data.append(cols)
            else:
                print('Audio file with no transcription: {}'.format(aud_fname))
                shutil.copy(file, tmp_au_without_transcripts.joinpath(aud_fname))
    df = pd.DataFrame(data)
    return df

In [137]:
df = document_the_data()

In [51]:
att = pd.read_csv("../data/chichewa/attributes.csv")

In [68]:
att = att[['gender', 'dialect', 'audio_filename']]

In [82]:
att['audio_filename'] = att.audio_filename.apply(lambda x: "{}.wav".format(x.split(".")[0]))

/var/folders/w3/dfvgjkh10wz8t573m_c20xf00000gp/T/ipykernel_53965/2863048343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att['audio_filename'] = att.audio_filename.apply(lambda x: "{}.wav".format(x.split(".")[0]))


In [138]:
df2 = df.merge(right=att, on='audio_filename', how='left')

In [139]:
df = df2[['age', 'gender', 'dialect', 'audio_filename', 'duration_sec',
       'transc_filename', 'collection_mode', 'num_speakers']]

In [149]:
df.columns

Index(['age', 'gender', 'dialect', 'audio_filename', 'duration_sec',
       'transc_filename', 'collection_mode', 'num_speakers'],
      dtype='object')

In [168]:
metadata = OUTPUT_AUDIO_DIR.parents[0].joinpath("dmatekenya_chichewa_metadata.csv")

In [163]:
df.to_csv(metadata, index=False)

In [169]:
df2 = pd.read_csv(metadata)

In [16]:
files = Path("/Users/dmatekenya/Downloads/andrew-recordings/")
outdir = Path("../audio-data-collection/temp-holding-before-sync/andrew-recordings-untranscribed/")

In [22]:
cnt = 1
tot_duration_sec = 0
for file in files.iterdir():
    audio_seg = AudioSegment.from_file(file)
    tot_duration_sec += len(audio_seg)//1000
    au_fname = "andrew_records_{}.wav".format(cnt)
    au_dst = outdir.joinpath(au_fname)
    AudioSegment.from_file(file).export(au_dst, format="wav")
    cnt += 1

In [19]:
asante_texts = Path("/Users/dmatekenya/Downloads/asante/")
asante_more_audios = Path("/Users/dmatekenya/Downloads/voicenotes//")
asante_outdir = Path.cwd().parents[0].joinpath("audio-data-collection/temp-holding-before-sync/from-asantee-may9/")
asante_outdir2 = Path.cwd().parents[0].joinpath("audio-data-collection/temp-holding-before-sync/from-asantee-may16/")

In [7]:
txt_files = [i for i in asante_texts.iterdir() if i.suffix == ".txt"]

In [8]:
txt_files[0]

PosixPath('/Users/dmatekenya/Downloads/asante/asante 219.txt')

In [11]:
for file in txt_files:
    fname = file.parts[-1]
    num = int(re.findall(r'\d+', fname)[0])
    new_fname = "asante_records_{}.txt".format(num)
    new_fpath = asante_outdir.joinpath(new_fname)
    shutil.move(file, new_fpath)

In [20]:
cnt = 378
for file in asante_more_audios.iterdir():
    new_fname = "asante_records_{}.wav".format(cnt)
    new_fpath = asante_outdir2.joinpath(new_fname)
    AudioSegment.from_file(file).export(new_fpath, format="wav")
    cnt += 1

In [21]:
x = AudioSegment.from_file(file)

In [23]:
f = asante_outdir2.joinpath('asante_records_378.wav')
aud = AudioSegment.from_file(f)

In [30]:
wau_outdir = Path.cwd().parents[0].joinpath("audio-data-collection/temp-holding-before-sync/audio-from-whatsapp-videos-may22")

In [ ]:
cnt = 1
for file in Path("/Users/dmatekenya/Downloads/whatsapp-vids/").iterdir():
    new_fname = "audio_from_whatsapp_vid_{}.wav".format(cnt)
    new_fpath = wau_outdir.joinpath(new_fname)
    clip = mp.VideoFileClip(str(file))
    clip.audio.write_audiofile(new_fpath, bitrate='50k')
    cnt += 1

In [35]:
clip = mp.VideoFileClip(str(file))
#clip.audio.write_audiofile("theaudio.mp3")

In [76]:
ind = Path("/Users/dmatekenya/Downloads/tmp-long3/")
outd = Path("../audio-data-collection/to-sync/long-form-without-transcripts-notshareable/")

In [77]:
cnt = 55
for file in ind.iterdir():
    try:
        new_fname = "untrascribed_social_research_interviews_{}.wav".format(cnt)
        new_fpath = outd.joinpath(new_fname)
        AudioSegment.from_file(file).export(new_fpath, format="wav")
        cnt += 1
    except:
        print(file)
        pass

In [104]:
ind = Path("/Users/dmatekenya/Downloads/dunstan-self-records")
wavs = [i.parts[-1][:-4] for i in ind.iterdir() if i.suffix == ".wav"]
txts = [i.parts[-1][:-4] for i in ind.iterdir() if i.suffix == ".txt"]
assert len(wavs) == len(txts)

In [58]:
for file in ind.iterdir():
    # if file.suffix == ".wav":
    #     fname = file.parts[-1][:-4]
    #     outfile = outd.joinpath(file.parts[-1])
    #     #print(outfile)
    #     if fname not in txts:
    #         print(fname)
    #         shutil.move(file,outfile)
    if file.suffix ==".txt":
        fname = file.parts[-1][:-4]
        outfile = outd.joinpath(file.parts[-1])
        #print(outfile)
        if fname not in wavs:
            print(fname)
            #shutil.move(file,outfile)

aud_from_whatsapp6


# Update the main data folder
The main data folder has audios and transcription files. Everytime we get new data, we will be updating this folder.

In [31]:
def check_audio_and_transc_filenames_match(ind):
    wavs = [i.parts[-1][:-4] for i in ind.iterdir() if i.suffix == ".wav"]
    txts = [i.parts[-1][:-4] for i in ind.iterdir() if i.suffix == ".txt"]
    
    print('='*50)
    print('AUDIOS WITHOUT MATCHING TXT FILE ....')
    print('='*50)
    needs_check = 0
    for wav in wavs:
        if wav not in txts:
            print(wav)
            needs_check += 1
    
    print()
    print('='*50)
    print('TXTS WITHOUT MATCHING AUDIO FILE ....')
    print('='*50)
    for txt in txts:
        if txt not in wavs:
            print(txt)
            needs_check += 1
    return needs_check

In [45]:
def sync_folder_to_audios(ind, out_data_dir, meta_dict, df_metadata, has_transcripts=True):
    """
    Takes any new folder with audios and appends the data to the main data folder
    """
    # check if all audios have matching text 
    df_meta = df_metadata.copy(deep = True)
    
    if has_transcripts:
        wavs = [i.parts[-1][:-4] for i in ind.iterdir() if i.suffix == ".wav"]
        txts = [i.parts[-1][:-4] for i in ind.iterdir() if i.suffix == ".txt"]
    else:
        wavs = [i.parts[-1][:-4] for i in ind.iterdir() if i.suffix == ".wav"]
    
    for file in wavs:
        try:
            # Move the files
            in_aud_fpath = ind.joinpath("{}.wav".format(file))
            out_aud_fpath = out_data_dir.joinpath("{}.wav".format(file))
            assert in_aud_fpath.exists()
            shutil.copy(in_aud_fpath, out_aud_fpath)
            if has_transcripts:
                in_txt_fpath = ind.joinpath("{}.txt".format(file))
                out_txt_fpath = out_data_dir.joinpath("{}.txt".format(file))
                assert in_txt_fpath.exists()
                shutil.copy(in_txt_fpath, out_txt_fpath)
        except AssertionError:
            _, _, tb = sys.exc_info()
            traceback.print_tb(tb) # Fixed format
            tb_info = traceback.extract_tb(tb)
            filename, line, func, text = tb_info[-1]
            print(file)
            print('An error occurred on line {} in statement {}'.format(line, text))
            continue
        
        # append row to metadata
        if has_transcripts:
            audio_seg = AudioSegment.from_file(in_aud_fpath)
            duration_sec = len(audio_seg)//1000
        
            meta_dict['duration_sec'] = duration_sec
            meta_dict['audio_filename'] = "{}.wav".format(file)
            meta_dict['transc_filename'] = "{}.txt".format(file)
        else:
            meta_dict['duration_sec'] = np.NaN
            meta_dict['audio_filename'] = "{}.wav".format(file)
            meta_dict['transc_filename'] = np.NaN
                
        df_meta = df_meta.append(meta_dict, ignore_index=True)
        
    # check that everything went well
    assert df_meta.shape[0] - df_metadata.shape[0] == len(wavs)
    
    return df_meta

## First check all folders with transcriptions

In [30]:
for file in Path("../audio-data-collection/to-sync/long-form-with-transcripts-notshareable").iterdir():
    if file.suffix == ".MP3":
        fname = file.parts[-1].split(".")[0]
        new_fpath = file.parents[0].joinpath("{}.wav".format(fname))
        print(new_fpath)
        AudioSegment.from_file(file).export(new_fpath, format="wav")

../audio-data-collection/to-sync/long-form-with-transcripts-notshareable/transcribed_social_research_interviews_6.wav
../audio-data-collection/to-sync/long-form-with-transcripts-notshareable/transcribed_social_research_interviews_7.wav
../audio-data-collection/to-sync/long-form-with-transcripts-notshareable/transcribed_social_research_interviews_5.wav


In [ ]:
folders_to_sync = Path("../audio-data-collection/to-sync/")
for folder in folders_to_sync.iterdir():
    if not folder.is_dir():
        continue
    parts = folder.parts[-1].split("-")
    print(folder.parts[-1])
    if "priscilla" in parts:
        continue
    if "without" in parts:
        continue
    else:
        needs_check = check_audio_and_transc_filenames_match(folder)
        if needs_check > 0:
            print('LETS FIX THESE FILES FIRST')
            break

## Update the data folder

In [65]:
df_base_meta = pd.read_csv("../data/chichewa/chich_speech_dataset_metadata.csv")
# add whether the data can be shared or not and whether it has a transcription
df_base_meta['shareable'] = 'Yes'
df_base_meta['has_transcript'] = 'Yes'

In [66]:
outd = Path("../data/chichewa/chich_speech_audio_files/")
meta_dunstan = {'age': '36-41', 
             'gender':'Male', 'dialect':'Nkhotakota'
             , 'audio_filename': None, 'duration_sec': None,
       'transc_filename':None, 'collection_mode': 'self-recording', 
             'num_speakers': 1, 'has_transcript': "Yes",'shareable': "Yes"}
dunstan = Path("../audio-data-collection/to-sync/dunstan-self-records-with-transcripts/")
df_meta_dunstan = sync_folder_to_audios(out_data_dir=outd, ind=dunstan,
                      meta_dict=meta_dunstan, df_metadata=df_base_meta, has_transcripts=True)

In [67]:
meta_asante = {'age': '24-29', 
             'gender':'Male', 'dialect':'Nkhotakota'
             , 'audio_filename': None, 'duration_sec': None,
       'transc_filename':None, 'collection_mode': 'self-recording', 
             'num_speakers': 1, 'has_transcript': "Yes",'shareable': "Yes"}
asante = Path("../audio-data-collection/to-sync/asante-with-transcripts/")
df_meta_asante = sync_folder_to_audios(out_data_dir=outd, ind=asante,
                      meta_dict=meta_asante, df_metadata=df_meta_dunstan, has_transcripts=True)

In [68]:
meta_andrew = {'age': '36-41', 
             'gender':'Male', 'dialect':'Ntchisi'
             , 'audio_filename': None, 'duration_sec': None,
       'transc_filename':None, 'collection_mode': 'self-recording', 
             'num_speakers': 1, 'has_transcript': "Yes",'shareable': "Yes"}
andrew = Path("../audio-data-collection/to-sync/andrew-self-recordings/")
df_meta_andrew = sync_folder_to_audios(out_data_dir=outd, ind=andrew,
                      meta_dict=meta_andrew, df_metadata=df_meta_asante, has_transcripts=True)

In [69]:
meta_whatsapp = {'age': np.NaN, 
             'gender':np.NaN, 'dialect':np.NaN
             , 'audio_filename': None, 'duration_sec': None,
       'transc_filename':None, 'collection_mode': 'whatsapp audios or videos', 
             'num_speakers': "1-3", 'has_transcript': "Yes",'shareable': "Yes"}
whatsapp = Path("../audio-data-collection/to-sync/whatsapp-with-transcripts/")
df_meta_whatsapp = sync_folder_to_audios(out_data_dir=outd, ind=whatsapp,
                      meta_dict=meta_whatsapp, df_metadata=df_meta_andrew, has_transcripts=True)

In [70]:
meta_long = {'age': np.NaN, 
             'gender':np.NaN, 'dialect':np.NaN
             , 'audio_filename': None, 'duration_sec': None,
       'transc_filename':None, 'collection_mode': 'interviews(FGD), social research', 
             'num_speakers': "1-3", 'has_transcript': "Yes",'shareable': "No"}
long_transc = Path("../audio-data-collection/to-sync/long-form-with-transcripts-notshareable/")
df_meta_long_transc = sync_folder_to_audios(out_data_dir=outd, ind=long_transc,
                      meta_dict=meta_long, df_metadata=df_meta_whatsapp, has_transcripts=True)

In [71]:
meta_long_no_transc = {'age': np.NaN, 
             'gender':np.NaN, 'dialect':np.NaN
             , 'audio_filename': None, 'duration_sec': None,
       'transc_filename':None, 'collection_mode': 'interviews(FGD), social research', 
             'num_speakers': "1-3", 'has_transcript': "No",'shareable': "No"}
long_no_transc = Path("../audio-data-collection/to-sync/long-form-without-transcripts-notshareable/")
df_meta_long_no_transc = sync_folder_to_audios(out_data_dir=outd, ind=long_no_transc,
                      meta_dict=meta_long_no_transc, df_metadata=df_meta_long_transc, has_transcripts=False)

In [72]:
meta_whatsapp_no_transc = {'age': np.NaN, 
             'gender':np.NaN, 'dialect':np.NaN
             , 'audio_filename': None, 'duration_sec': None,
       'transc_filename':None, 'collection_mode': 'whatsapp audios or videos', 
             'num_speakers': "1-3", 'has_transcript': "No",'shareable': "Yes"}
whatsapp_no_transc = Path("../audio-data-collection/to-sync/whatsapp-without-transcripts/")
df_meta_whatsapp = sync_folder_to_audios(out_data_dir=outd, ind=whatsapp_no_transc,
                      meta_dict=meta_whatsapp_no_transc, df_metadata=df_meta_long_no_transc, has_transcripts=False)

In [29]:
tmp = Path("../audio-data-collection/to-sync/priscilla/")
file_lst = [i for i in tmp.iterdir()]

for file in file_lst:
    fname = file.parts[-1]
    if " " in fname:
        new_fname = fname.replace(" ", "")
        new_fpath = "../audio-data-collection/to-sync/priscilla-self-records/{}".format(new_fname)
        shutil.move(file, new_fpath)
    else:
        new_fpath = "../audio-data-collection/to-sync/priscilla-self-records/{}".format(fname)
        shutil.move(file, new_fpath)

In [ ]:
folders_to_sync = Path("../audio-data-collection/to-sync/")
for folder in folders_to_sync.iterdir():
    if not folder.is_dir():
        continue
    parts = folder.parts[-1].split("-")
    print(folder.parts[-1])
    if "priscilla" in parts:
        continue
    if "without" in parts:
        continue
    else:
        needs_check = check_audio_and_transc_filenames_match(folder)
        if needs_check > 0:
            print('LETS FIX THESE FILES FIRST')
            break

In [39]:
folder =  Path("../audio-data-collection/to-sync/priscilla-self-records/")
needs_check = check_audio_and_transc_filenames_match(folder)

AUDIOS WITHOUT MATCHING TXT FILE ....

TXTS WITHOUT MATCHING AUDIO FILE ....


In [40]:
df_base_meta = pd.read_csv("../data/chichewa/metadata.csv")

In [42]:
outd = Path("../data/chichewa/chich_speech_audio_files/")

In [46]:
meta_priscilla = {'age': 20, 
             'gender':'female', 'dialect':'Zomba'
             , 'audio_filename': None, 'duration_sec': None,
       'transc_filename':None, 'collection_mode': 'whatsapp audios or videos', 
             'num_speakers': "1", 'has_transcript': "Yes",'shareable': "Yes"}
priscilla = Path("../audio-data-collection/to-sync/priscilla-self-records/")
df_meta_whatsapp = sync_folder_to_audios(out_data_dir=outd, ind=priscilla,
                      meta_dict=meta_priscilla, df_metadata=df_base_meta, has_transcripts=True)

In [48]:
df_meta_whatsapp.to_csv("../data/chichewa/metadata.csv", index=False)

In [49]:
df_meta_whatsapp

,age,gender,dialect,audio_filename,duration_sec,transc_filename,collection_mode,num_speakers,shareable,has_transcript
0,42-47,Male,NaN,dunstan_on_march_26.wav,4.0,dunstan_on_march_26.txt,Spontaneous,1-2,Yes,Yes
1,36-41,Male,NaN,onesmus.wav,30.0,onesmus.txt,Spontaneous,1-2,Yes,Yes
2,24-29,Male,NaN,WAU1.wav,12.0,WAU1.txt,Spontaneous,1-2,Yes,Yes
3,24-29,Male and Female,NaN,WAU2.wav,178.0,WAU2.txt,Spontaneous,1-2,Yes,Yes
4,30-35,Female,NaN,WAU3.wav,191.0,WAU3.txt,Spontaneous,1-2,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...
2628,20,female,Zomba,priscilla_self_records579.wav,10.0,priscilla_self_records579.txt,whatsapp audios or videos,1,Yes,Yes
2629,20,female,Zomba,priscilla_self_records237.wav,4.0,priscilla_self_records237.txt,whatsapp audios or videos,1,Yes,Yes
2630,20,female,Zomba,priscilla_self_records551.wav,4.0,priscilla_self_records551.txt,whatsapp audios or videos,1,Yes,Yes
2631,20,female,Zomba,priscilla_self_records545.wav,6.0,priscilla_self_records545.txt,whatsapp audios or videos,1,Yes,Yes
